## Sorting and Grouping
The Search service offers the ability to organize your result set in different ways.  Whether you choose to sort based on a specific property, group properties or boost the result set, you have the option to override the default ordering returned. In addition, you can also use navigation to organize your buckets of data returned.

By default, the result set will provide some basic ranking, depending on the type of query or filter performed.  

#### NuGet Packages

In [1]:
#r "nuget:Refinitiv.DataPlatform.Content, 1.0.0-alpha3"
#r "nuget:Microsoft.Data.Analysis"

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed package Refinitiv.DataPlatform.Content version 1.0.0-alpha3

Installed package Microsoft.Data.Analysis version 0.4.0

In [2]:
using Newtonsoft.Json.Linq;
using Refinitiv.DataPlatform.Content.SearchService;
using Refinitiv.DataPlatform.Core;
using Microsoft.Data.Analysis;
using Microsoft.AspNetCore.Html;
using System.Linq;
using System;

#### Table Output
Helper routine to output data in a table format.

In [3]:
Formatter.Register<IList<JObject>>((hits, writer) =>
{
    if (hits.Count > 0)
    {
        var fields = new List<String>();
        var rows = new List<ICollection<IHtmlContent>>();
        var data = new Dictionary<string, IHtmlContent>();

        foreach(var hit in hits)
        {
            var cells = new List<IHtmlContent>();
            foreach (var val in hit.Properties())
            {
                if ( !fields.Any(item => item.Equals(val.Name)) )
                    fields.Add(val.Name);
                data[val.Name] = td(val.Value.ToString());
            }
            rows.Add(new List<IHtmlContent>(data.Values));
            data.Keys.ToList().ForEach(x => data[x] = td(""));
        }
        
        var headers = new List<IHtmlContent>();
        headers.AddRange(fields.Select(c => (IHtmlContent)th(c)));

        var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
        writer.Write(t);
    }
}, "text/html");

In [4]:
// Create a session into the desktop
var session = CoreFactory.CreateSession(new DesktopSession.Params()
                            .AppKey("Your API Key here")
                            .OnState((s, state, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (State: {state})"))
                            .OnEvent((s, eventCode, msg) => Console.WriteLine($"{DateTime.Now}:{msg}. (Event: {eventCode})")));
session.Open();

17/12/2020 12:35:04 PM:Session is Pending. (State: Pending)
17/12/2020 12:35:04 PM:{
  "Contents": "Desktop Session Successfully Authenticated"
}. (Event: SessionAuthenticationSuccess)
17/12/2020 12:35:04 PM:Session is Opened. (State: Opened)


#### Example - Default ranking and how to boost results
To demonstrate default sorting, I'm going to search for documents that match a specific ticker. This will result in the documents ranked based on the relevancy and scores assigned to each document.

Search for Vodafone (VOD) against the ticker:

In [5]:
var response = Search.Definition(Search.View.SearchAll).Filter("TickerSymbol eq 'VOD'")
                                                       .Select("_, RCSExchangeCountryLeaf")
                                                       .GetData();
response.Data.Hits

QUOTExEQUITY,"Vodafone Group PLC, Ordinary Share, London Stock Exchange",55836053361,1049261,VOD.L,United Kingdom
QUOTExEQUITY,"Vodacom Group Ltd, Ordinary Share, Johannesburg Stock Exchange",55849756323,59938825,VODJ.J,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group Limited Cash Settled SSF Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21508525903,113268320,VODSc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group CFD Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21600821292,244233208,VODCFDc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Can Do VODX Vodacom Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21534036541,148452798,VODXFLc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group Limited Cash Settled CFD Sabor Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21693141305,363600087,VODSCFDc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group Ltd SSF Cash Settled Dividend Neutral Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21707726176,381845837,VODTc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodafone Group PLC Dividend Neutral Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21481060885,53826719,VODDc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodafone Group PLC IDX Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21481060918,53826752,VODGc1,South Africa
QUOTExEQUITY,"Johannesburg Stock Exchange Vodacom Group SSF Equity Future Continuation 1, Equity Future, ZAR, Johannesburg Stock Exchange",21481084690,60188651,VODQc1,South Africa


The above output ranked the common share 'VOD.L', listed within the UK, at the top.  This is largely due to the fact that 'VOD.L' is the most significant or liquid asset in the list.  However, if I decide to override this default ranking by specifying that I want to show the listed documents within the Unitied States at the top, I can do this 
by applying a *Boost* parameter.

In [6]:
response = Search.Definition(Search.View.SearchAll).Filter("TickerSymbol eq 'VOD'")
                                                   .Boost("RCSExchangeCountryLeaf eq 'United States'")
                                                   .Select("_, RCSExchangeCountryLeaf")
                                                   .GetData();
response.Data.Hits

QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NASDAQ Global Select Consolidated",55839401038,726233,VOD.O,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NASDAQ Stock Exchange Global Select Market",55835364259,1092283,VOD.OQ,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Cboe EDGX Exchange - Nasdaq Global Select Market",21475045362,73238578,VOD.DG,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Cboe BZX Exchange - Nasdaq Global Select Market",55845638958,53705532,VOD.Z,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Boston SE when trading NASDAQ Global Select Market",55846134804,55957391,VOD.B,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NYSE Arca when trading NASDAQ Global Select Market",55837254988,754172,VOD.P,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Cboe BYX Exchange - Nasdaq Global Select Market",21475122433,76383634,VOD.ZY,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, Cboe EDGA Exchange - Nasdaq Global Select Market",21475045135,73240457,VOD.DY,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NASDAQ OMX PHLX Global Select Market",21475115114,76271998,VOD.PH,United States
QUOTExEQUITY,"Vodafone Group PLC, Depository Receipt, NYSE National Exchange when trading Nasdaq Global Select Market",55838271132,735992,VOD.C,United States


#### Example - List the youngest CEO's.
The order_by parameters will sort, ascending (default) or descending, based on the birth year property.

Note: In the following example, not every document that identifies a CEO will have a reported birth year. Because we are sorting based on the year they were born, 
all CEO's where the year of birth is not recorded will be bumped to the bottom of the list.

In [7]:
var filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')";
var select = "FullName, YearOfBirth, DTCharacteristics, PrimaryOfficerDirectorRIC, PrimaryOfficerDirector";

response = Search.Definition(Search.View.People).Filter(filter)
                                                .OrderBy("YearOfBirth desc, LastName, FirstName")
                                                .Select(select)
                                                .GetData();
response.Data.Hits

Grigory Sergeevich Sandulov,1991,"General Director (CEO), Director",OMRPI.RTS,Omskiy Rechnoy Port AO
Yehonatan Shachar,1991,Chief Executive Officer,CHR.TA,Chiron Refineries Ltd
Myeong Jin Chae,1990,"Chief Executive Officer, Director",058220.KQ,Arion Technology Inc
Pedro de Godoy Bueno,1990,"Chief Executive Officer, Director",DASA3.SA,Diagnosticos da America SA
Hikaru Ohno,1990,"President, Chief Executive Officer, Representative Director",4493.T,Cyber Security Cloud Inc
Vinit Agrawal,1989,Chief Executive Officer,LIBD.BO,Libord Securities Ltd
Erik Gatenholm,1989,"Chief Executive Officer, Director",CLNKb.ST,Cellink AB
Yasuyuki Kawabata,1989,"Chief Executive Officer, Representative Director",7352.T,Branding Engineer Co Ltd
Jack O.A. Lundin,1989,"Chief Executive Officer, Director",BSR.V,Bluestone Resources Inc
Nobuhiro Moriyoshi,1989,"Chief Executive Officer, Representative Director",3989.T,Sharingtechnology Inc


Instead of sorting by the year of birth, let's organize the output so the company is grouped.

**Note**: While we can see the company's grouped, we can also observe the many of the entries do not have a year of birth recorded.

In [8]:
filter = "OfficerDirector(RoleTitleCode eq 'CEO' and RoleStatus eq 'Active')";
select = "FullName, YearOfBirth, DTCharacteristics, PrimaryOfficerDirectorRIC, PrimaryOfficerDirector";

response = Search.Definition(Search.View.People).Filter(filter)
                                                .GroupBy("PrimaryOfficerDirector")
                                                .Top(20)
                                                .Select(select)
                                                .GetData();
response.Data.Hits

FullName DTCharacteristics PrimaryOfficerDirectorRIC PrimaryOfficerDirector YearOfBirth John Wendell Thompson Independent Non-Executive Chairman of the Board MSFT.O Microsoft Corp Satya Nadella Chief Executive Officer, Director MSFT.O Microsoft Corp Art D. Levinson Independent Chairman of the Board AAPL.O Apple Inc 1950 Tim D. Cook Chief Executive Officer, Director AAPL.O Apple Inc Jeff P. Bezos Chairman of the Board, President, Chief Executive Officer AMZN.O Amazon.com Inc Keith Brian Alexander Director AMZN.O Amazon.com Inc Jon J. Rubinstein Independent Director AMZN.O Amazon.com Inc 1957 Sundar Pichai Chief Executive Officer, Director GOOGL.O Alphabet Inc 1972 Roger W. Ferguson Independent Director GOOGL.O Alphabet Inc Mark Zuckerberg Chairman of the Board, Chief Executive Officer, Founder FB.O Facebook Inc Adam D'Angelo Venture Economics Executive FB.O Facebook Inc Dustin A. Moskovitz Venture Economics Executive FB.O Facebook Inc Warren Edward Buffett Chairman of the Board, Chief Executive Officer BRKa Berkshire Hathaway Inc 1930 Gregory E. Abel Vice Chairman � Non Insurance Operations, Director BRKa Berkshire Hathaway Inc Steve B. Burke Independent Director BRKa Berkshire Hathaway Inc 1959 Daniel Zhang Chairman of the Board, Chief Executive Officer BABA.K Alibaba Group Holding Ltd 1972 Fan Jiang President - Taobao and Tmall BABA.K Alibaba Group Holding Ltd Huateng Ma Executive Chairman of the Board, Chief Executive Officer 0700.HK Tencent Holdings Ltd Al Francis Kelly Chairman of the Board, Chief Executive Officer V Visa Inc Alex Gorsky Chairman of the Board, Chief Executive Officer JNJ Johnson & Johnson

#### Example - Sorting using Navigators
By default, when you use a navigator against a property, it will sort all results based on the number of matches for each value within a bucket.  For example, if I were to list the top 10 exchanges within Canada, we can see the count value ranked, indicating the number instruments matched on that exchange.

In [9]:
response = Search.Definition(Search.View.EquityQuotes).Filter("RCSExchangeCountryLeaf eq 'Canada'")
                                                      .Top(0)
                                                      .Navigators("ExchangeName(buckets:10)")
                                                      .GetData();
Console.WriteLine(response.Data.Navigators["ExchangeName"]["Buckets"]);

[
  {
    "Label": "Montreal Options Exchange",
    "Count": 45512
  },
  {
    "Label": "Montreal Exchange",
    "Count": 25230
  },
  {
    "Label": "The Toronto Stock Exchange",
    "Count": 11460
  },
  {
    "Label": "Nasdaq CXC",
    "Count": 9821
  },
  {
    "Label": "Canadian Securities Exchange/PURE - CSE Listed & Other Canadian Listed Securities",
    "Count": 8100
  },
  {
    "Label": "Canadian Composite Quote/Trade",
    "Count": 7809
  },
  {
    "Label": "TSX Venture Exchange (former Canadian Ventures Exchange)",
    "Count": 7103
  },
  {
    "Label": "Alpha - Canadian ATS",
    "Count": 7020
  },
  {
    "Label": "Omega - Candian ATS",
    "Count": 7001
  },
  {
    "Label": "Nasdaq CX2",
    "Count": 6540
  }
]


Using the above example, I can instead choose to sort based on the average daily volume within the exchange.  The following search will result in the top 5 Canadian exchanges, ranked based on the 90 day average volume:

In [10]:
response = Search.Definition(Search.View.EquityQuotes).Filter("RCSExchangeCountryLeaf eq 'Canada'")
                                                      .Top(0)
                                                      .Navigators("ExchangeName(buckets:5, desc:sum_AvgVol90D)")
                                                      .GetData();

In [33]:
// Pretty display of the listing
var writer = new System.IO.StringWriter();
var headers = new List<IHtmlContent> {th("Exchange"), th("Average 90-day Volume")};
var rows = new List<IList<IHtmlContent>>();
        
foreach (var entry in response.Data.Navigators["ExchangeName"]["Buckets"])
{
    var vol = entry["sum_AvgVol90D"].ToObject<long>();
    var cells = new List<IHtmlContent> {td(entry["Label"].ToString()), td($"{vol:n0}")};
    rows.Add(cells);
}
var t = table(thead(headers), tbody(rows.Select(r => tr(r))));
t

Canadian Composite Quote/Trade,"1,871,211,973"
The Toronto Stock Exchange,"1,048,732,175"
TSX Venture Exchange (former Canadian Ventures Exchange),"430,539,757"
Canadian Securities Exchange - CSE Listed,"134,195,786"
Canadian Securities Exchange/PURE - CSE Listed & Other Canadian Listed Securities,"50,373,942"
